In [2]:
import numpy as np
from sympy  import Symbol,Add,Mul,Pow,Number,symbols
from sympy import  factor,srepr,sympify,expand,sqrt,simplify
import matplotlib.pyplot as plt

In [3]:
def create_dictionary(m):
    m_bin= bin(m)[2:][::-1]
    m_dict={}
    for i,value in enumerate(m_bin):
        m_dict[i]= int(value)
    p_dict={}
    p_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        p_dict[i]= Symbol('p'+str(i))
    p_dict[int(np.ceil(len(m_bin)/2))-1]=1 # length half of m is being held.here need some generializtion to check out bit length of p and q optimum.

    q_dict={}
    q_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        q_dict[i]= Symbol('q'+str(i))
    q_dict[int(np.ceil(len(m_bin)/2))-1]=1

    n=  len(m_bin) + int(np.ceil(len(m_bin)/2)) - 1
    z_dict={}
    for i in range(n):
        for j in range(i+1):
            if i != j:
                if i >= len(m_bin) :
                    pass
                elif j ==0 :
                    pass
                else :
                    if i-j >= int(np.ceil(len(m_bin)/2))-1  :
                        z_dict[(j, i)] = 0
                        
                    else:
                        z_dict[(j, i)] =  Symbol('z'+str(j)+str(i))

        z_dict[(len(m_bin)-4, len (m_bin)-1)] =0
    return m_dict , p_dict ,q_dict , z_dict

In [4]:
def max_sum(clause):
    max=0
    if clause.func == Add :
        for t in clause.args :
            if   isinstance(t, Number) :
                max = max+ int(t)
            #elif isinstance(t, Number) and int(t) < 0:
            #    max -= int(t) 
                #print(max)
            
            if t.func == Mul :
                if  isinstance(t.args[0], Symbol)  :           ###changed  from  previous one
                    max= max+1

                if isinstance(t.args[0], Number) and t.args[0] > 0:
                    max = max + (int(t.args[0]))
                    
                if isinstance(t.args[0], Number) and t.args[0] < 0:
                    max = max + 0
                      
                     
                    
                if isinstance(t.args[0], Number) and  t.args[0] <0 :
                    
                    pass  
            if t.func == Symbol  :
                
                max=max+1
               
            #if isinstance(t ,Symbol):
             #   max=max+1
            
            '''if   isinstance(t, Number) :
                max = max+ int(t)
            #elif isinstance(t, Number) and int(t) < 0:
            #    max -= int(t) '''
              
                

            #else :
             #   max = max+1
      

    elif clause.func == Mul :
        if  isinstance(clause.args[0], Number) and  clause.args[0] >0 :
            max =max+ int(clause.args[0])
        if   isinstance(clause.args[0], Number) and  clause.args[0] <0 :
            max =max+ int(0)
            
        else :
            max=max+1

    elif clause.func == Symbol :
        max = 1
    elif isinstance(clause,Number):
        max=max+int(clause)
   
    return max

In [5]:
def simplify_clause(clause, equation,i):
    
    clause=clause.subs(equation).expand()
    

    if clause.func == Add:
        for t in clause.args:
            if t .func == Mul and 'Pow' in  srepr(t):
                for s in t.args:
                    if s.func  == pow:
                        clause=clause.subs({s: s.args[0]})
            if t.func == pow :
                clause=clause -t +t.args[0]

            else :
                f_clause = factor(clause)
                if f_clause .func == Mul :
                    if isinstance(f_clause.args[0],Number):
                        clause = clause /f_clause.args[0]
    if i == 1:
        equation= rule_1(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    if i == 2:
        equation= rule_2(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    
    if i == 3:
        equation= rule_3(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    if i == 4:
       equation= rule_4(clause,equation)
       clause=clause.subs(equation).expand()
       num=max_sum(clause)
       #print(clause,equation)

    if i == 5:
       equation= rule_5(clause,equation)
       clause=clause.subs(equation).expand()
       num=max_sum(clause)
    
     #  print(clause,equation)
    #equation=rule_3(clause,equation)
    #clause=clause.subs(equation).expand()
    

    return  clause ,equation

In [6]:
def create_dictionary_robust(m):
    m_bin=bin(m)[2:][::-1]
    m_dict={}
    for i,j in enumerate(m_bin):
        m_dict[i] =int (j)

    p_dict={}
    p_dict[0]=1
    
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        p_dict[i]= Symbol('p'+str(i))
    p_dict[int(np.ceil(len(m_bin)/2))-1]=1 # length half of m is being held.here need some generializtion to check out bit length of p and q optimum.

    q_dict={}
    q_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        q_dict[i]= Symbol('q'+str(i))
    q_dict[int(np.ceil(len(m_bin)/2))-1]=1

    n=  len(m_bin) + int(np.ceil(len(m_bin)/2)) - 1
    z_dict1={}
    z_dict2={}

    for i in range(2,len(m_bin)):
        for j in range(i-int(np.floor(np.log2(i))),i):
            if i!= j:
                z_dict1[(j, i)] =  Symbol('z'+str(j)+str(i))

    for i in range(1,len(m_bin)-1):
        for j in range(i,i+len(bin(i)[2:][::-1])+1):
            if i!= j  and j <= len(m_bin):
                z_dict2[(i, j)] =  Symbol('z'+str(i)+str(j))




    return m_dict , p_dict ,q_dict , z_dict2
                



    


In [7]:
def create_clause1(m,p,q,z):
    clauses=[]
    clauses1=[]
    z_zero={}
    n=len(m)+int(np.ceil(len(m)/2)) -1
    for i in range(n):
        clause=0
        for j in range(i+1):
            clause+=q.get(j,0) * p.get(i-j,0)
        #clause  += - m.get(i,0)
        for j in range(i+1):
            clause += z.get((j, i), 0)
        
        if type(clause) == int:
            clause = sympify(clause)
        if clause != 0:
            max_sum1=max_sum(clause)
            if max_sum1 !=0  :
                max_carry = int(np.floor(np.log2(max_sum1)))
            else : 
                max_carry  = 0
        for j in range(len(z)):
            if j-i > max_carry:
                if z.get((i-1, j), 0) != 0:
                    z[(i-1, j)] = 0
                    z_zero['z'+str(i-1)+str(j)] = 0 
        clause  += - m.get(i,0)
        for j in range(i+1, i+5):
            if j-i <=  max_carry +1 :
               clause += - 2**(j-i) * z.get((i, j), 0) 
        
        
        if clause ==0 :
            clause = sympify(clause)
        clauses.append(clause)

    #for clause in clauses:
     #   for keys in z_zero:
     #       clause=str(clause).replace(z_zero[keys],'0')
     #   clauses1.append(clause)
            

    return clauses

In [8]:
def rule_21(clause, expression):
    if clause.func == Add and len(clause.args) ==3 and len(list(clause.free_symbols))==2 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x'),list(clause.free_symbols)[1]: Symbol('y')})
        rule= Symbol('x') + Symbol('y') -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0] * list(clause.free_symbols)[1]] =0
            expression[list(clause.free_symbols)[0] + list(clause.free_symbols)[1]] =1
            '''if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[lisit(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]'''
    if (len(expression)!= 0):
        print(('rule21applied'))
    
    return expression

In [9]:
def rule_31(clause, expression):
    if clause.func == Add and len(clause.args) ==2 and len(list(clause.free_symbols))==1 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')  -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]] =1
            '''if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[list(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]'''
    if (len(expression)!= 0):
        print(('rule31applied'))
    
    return expression

In [10]:
def rule_11(clause,expression):
    negative =[]
  
    for t in clause.args :
        if t.func == Mul  and isinstance(t.args[0], Number)  and t.args[0]<0 :
            negative.append(t)
        

        
    if len(negative) >0 :
        for  t in negative:
            if   -t.args[0] > max_sum(clause):
                var= t/t.args[0]
                expression[var] =0
    if (len(expression)!= 0):
        print(('rule11applied'))
    
    #print (max_sum(clause))

    return expression

In [11]:
def rule_51(clause, known_expressions):
    
    if clause.func == Add and len(clause.args) == 2:
        if len(clause.args[0].free_symbols) == 0:
            constant_a = clause.args[0]
            if clause.args[1].func == Mul:
                constant_b = clause.args[1].args[0]
                symbol = clause.args[1] / constant_b
                if isinstance(constant_a,Number) and isinstance(constant_b,Number):
                    if constant_a > 0 or constant_b < 0:
                    
                       known_expressions[symbol] = 1
    return known_expressions

In [12]:
def  rule_41(clause , expression):
    count=0
    if clause.func == Add and max_sum(clause) == len(list(clause.free_symbols)):
        for t in clause.args :
            if t.func != Mul :
                expression[t] =0
    
    else :
        for t in clause.args :
            if isinstance(t,Number):
                count= count+t
        if clause.func == Add and  len(list(clause.free_symbols)) ==- count :
            for t in clause.args :
                if t != count :
                    expression[t] = 1

In [13]:
def  rule_61(clause,expression):
    if clause.func == Symbol   and  len(list(clause.free_symbols)) == 1:
        '''if isinstance(clause.args[0],Number):
            var= clause / clause.args[0]
            sub_clause=var.subs({list(clause.free_symbols)[0]: Symbol('x')})
            rule= Symbol('x')
            if sub_clause -rule ==0 :
               expression[list(clause.free_symbols)[0]] =0'''

        print(clause.free_symbols)
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')
        if sub_clause -rule ==0 :
           expression[list(clause.free_symbols)[0]] =0

    if clause.func == Mul  and   len(list(clause.free_symbols)) == 1: 
        var= clause / clause.args[0]
        sub_clause=var.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]] =0


In [14]:
def rule_71(clause,expression):
    count=0
    constant =0
    negative=[]
    if clause.func==Add  :
        for t in clause.args:
            if isinstance(t, Number) and int(t) > 0:
                constant= constant + int(t)
                print(type(int(t)))
            if  isinstance(t,Symbol):
                count = count +1
                print(type(count))
            if t.func==Mul : 
                if isinstance(t.args[0],Number) and  t.args[0] < 0:

                    negative.append(- t.args[0])
                else :

                    count =count +1
                
        print(negative)


    if len(negative) > 0 and constant + count == max(negative  ) and  int(count)  > 0:
        for t in clause.args:
            if t.func == Mul  and isinstance(t.args[0] , Number) :
                var=t/t.args[0]
                expression[var] = 1
                for i in var.args:
                    expression[i] =1
            if t.func == Mul and isinstance(t.args[0], Symbol):
                for i in t.args :
                    if isinstance(i, Symbol):
                       expression[i]=1

    
            
    return expression

In [15]:
m,p,q,z = create_dictionary_robust(3599)
p2=create_clause1(m,p,q,z)
p2


[0,
 p1 + q1 - 2*z12 - 1,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p1 + p2*q4 + p3*q3 + p4*q2 + q1 + z46 + z56 - 2*z67 - 4*z68 - 8*z69,
 p2 + p3*q4 + p4*q3 + q2 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 8*z811 - 2*z89,
 p4 + q4 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 1,
 -2*z1011 - 4*z1012 + z710 + z810 + z910,
 z1011 + z811 + z911 - 1,
 z1012 + z912,
 0,
 0,
 0,
 0]

In [16]:
expression_3599={}
renew_clause_1=[]
for clauses in p2:
    
    rule_11(clauses,expression_3599)

for clauses in  p2:
    renew_clause_1.append(clauses.subs(expression_3599).expand())
renew_clause_1





rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied


[0,
 p1 + q1 - 1,
 p1*q1 + p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p1 + p2*q4 + p3*q3 + p4*q2 + q1 + z46 + z56 - 2*z67 - 4*z68,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 4*z911 - 1,
 -2*z1011 + z810 + z910,
 z1011 + z911 - 1,
 0,
 0,
 0,
 0,
 0]

In [17]:
renew_clause_2=[]
for clauses in renew_clause_1:
    
    rule_21(clauses,expression_3599)

for clauses in  renew_clause_1:
    renew_clause_2.append(clauses.subs(expression_3599).expand())
renew_clause_2



rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied


[0,
 0,
 p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 4*z911 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [19]:
renew_clause_3=[]
for clauses in renew_clause_2:
    
    rule_11(clauses,expression_3599)

for clauses in  renew_clause_2:
    renew_clause_3.append(clauses.subs(expression_3599).expand())
renew_clause_3


rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied


[0,
 0,
 p2 + q2 - 1,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]